In [9]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import glob
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score
from sklearn import svm
from sklearn.model_selection import train_test_split
import joblib

from ipynb.fs.full.hog_config import hog_human, WIN_SIZE

In [10]:
datasets = [
        (1, "dataset/human_pos"),
        (0, "dataset/human_neg")
         ]

In [11]:
features = np.zeros((1, 216), np.float32)
labels = np.zeros(1, np.int64)

for ds in datasets:
    label = ds[0]
    path = ds[1]
    image_files = glob.glob(path + "/*")
    for image_file in image_files:
        img = cv2.imread(image_file, cv2.IMREAD_COLOR)
        resized_im = cv2.resize(img, WIN_SIZE)
        descriptor = np.transpose(hog_human.compute(resized_im))
        features = np.vstack((features, descriptor))
        labels = np.vstack((labels, label))

features = np.delete(features, (0), axis=0)
labels = np.delete(labels, (0), axis=0).ravel()

In [12]:
x_train, x_test, y_train, y_test = train_test_split(
    features, 
    labels, 
    test_size=0.2, 
    random_state=42
)

In [13]:
print("== train set ==\n", x_train.shape, y_train.shape)
print('== test set ==\n', x_test.shape, y_test.shape)

clf = svm.SVC()
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

print('=== RESULT ===')
print('Accuracy: ', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

joblib.dump(clf, 'model_hog_svm.pkl') 

== train set ==
 (2741, 216) (2741,)
== test set ==
 (686, 216) (686,)
=== RESULT ===
Accuracy:  0.956268221574344
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       236
           1       0.97      0.97      0.97       450

    accuracy                           0.96       686
   macro avg       0.95      0.95      0.95       686
weighted avg       0.96      0.96      0.96       686



['model_hog_svm.pkl']